In [24]:
import numpy as np
import os, sys
sys.path.append('../')
from data_functions import degree2radians, mapping_map_to_sphere, process_df, get_XYZV, create_spherical_heatmap_vals, get_lines
import pandas as pd
df = pd.read_csv("../data/GlobalLandTemperaturesByCity.csv")
df = df[df['AverageTemperature'].notna()]
df['Year'] = np.around([int(dt.split('-')[0]) for dt in df['dt']], -1)
tdf = process_df(df, ['Country'], 'Latitude', 'Longitude')

In [25]:
tdf = df.groupby(['Latitude', 'Longitude', 'Country', 'Year', 'X','Y','Z']).mean().reset_index()
tdf.head()

,Latitude,Longitude,Country,Year,X,Y,Z,AverageTemperature,AverageTemperatureUncertainty
0,-52.24,-70.95,Chile,1860,0.199869,-0.578819,-0.790583,5.368887,1.834042
1,-52.24,-70.95,Chile,1870,0.199869,-0.578819,-0.790583,5.572269,1.757287
2,-52.24,-70.95,Chile,1880,0.199869,-0.578819,-0.790583,5.413420,1.704214
3,-52.24,-70.95,Chile,1890,0.199869,-0.578819,-0.790583,5.299393,1.718598
4,-52.24,-70.95,Chile,1900,0.199869,-0.578819,-0.790583,5.498280,1.337174


In [26]:
import pycountry
keys = [list(pycountry.countries)[i].alpha_3 for i in range(len(pycountry.countries))]
names = [list(pycountry.countries)[i].name for i in range(len(pycountry.countries))]
conv = { v:k for k,v in zip(keys,names) }
def match_name(x):
    try:
        return conv[x]
    except KeyError as e:
        for  n in names:
            if x in n:
                return conv[n]
        return 'N/A'
tdf['codes'] = tdf.apply(lambda x: match_name(x['Country']), axis=1)

In [28]:
tdf.head()

,Latitude,Longitude,Country,Year,X,Y,Z,AverageTemperature,AverageTemperatureUncertainty,codes
0,-52.24,-70.95,Chile,1860,0.199869,-0.578819,-0.790583,5.368887,1.834042,CHL
1,-52.24,-70.95,Chile,1870,0.199869,-0.578819,-0.790583,5.572269,1.757287,CHL
2,-52.24,-70.95,Chile,1880,0.199869,-0.578819,-0.790583,5.413420,1.704214,CHL
3,-52.24,-70.95,Chile,1890,0.199869,-0.578819,-0.790583,5.299393,1.718598,CHL
4,-52.24,-70.95,Chile,1900,0.199869,-0.578819,-0.790583,5.498280,1.337174,CHL


In [34]:
import plotly.graph_objects as go

years = tdf['Year'].unique()[::5]
years.sort()
years_sliders = []

for year in years:
    ttdf = tdf[tdf['Year'] == year]

    years_sliders.append(go.Choropleth(
        visible=False,
        locations = ttdf['codes'],
        z = ttdf['AverageTemperature'],
        text = ttdf['Country'],
        colorscale = 'RdBu',
        reversescale=True,
        marker_line_color='darkgray',
        marker_line_width=0.5,
        colorbar_ticksuffix = '°C',
        colorbar_title = 'Temperature',
        zmin=df['AverageTemperature'].min(), zmax=df['AverageTemperature'].max(), zmid=0
    ))

fig = go.Figure(data=years_sliders)

steps = []
for i in range(len(years)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Year: " + str(years[i])}],
              label=str(years[i])  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=len(years),
    currentvalue={"prefix": "Steps: "},
    pad={"t": 50},
    steps=steps,
)]

geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='orthographic'
    )
fig.update_layout(
    sliders=sliders,
    geo=geo
)
fig.data[0].visible = True
fig.write_html('../plots/complex_globe.html')

fig.show()